# DenseNet

In [1]:
import tensorflow as tf
from utils import cifar10_input 

In [2]:
batch_size = 64
train_imgs, train_labels, val_imgs, val_labels = cifar10_input.load_data(image_size=96, batch_size=batch_size)

Instructions for updating:
Queue-based input pipelines have been replaced by `tf.data`. Use `tf.data.Dataset.from_tensor_slices(string_tensor).shuffle(tf.shape(input_tensor, out_type=tf.int64)[0]).repeat(num_epochs)`. If `shuffle=False`, omit the `.shuffle(...)`.
Instructions for updating:
Queue-based input pipelines have been replaced by `tf.data`. Use `tf.data.Dataset.from_tensor_slices(input_tensor).shuffle(tf.shape(input_tensor, out_type=tf.int64)[0]).repeat(num_epochs)`. If `shuffle=False`, omit the `.shuffle(...)`.
Instructions for updating:
Queue-based input pipelines have been replaced by `tf.data`. Use `tf.data.Dataset.from_tensors(tensor).repeat(num_epochs)`.
Instructions for updating:
To construct input pipelines, use the `tf.data` module.
Instructions for updating:
To construct input pipelines, use the `tf.data` module.
Instructions for updating:
Queue-based input pipelines have been replaced by `tf.data`. Use `tf.data.FixedLengthRecordDataset`.
Instructions for updating:
Q

### 构建基本卷积单元

In [3]:
import tensorflow.contrib.slim as slim

In [4]:
def bn_relu_conv(x, out_depth, scope='dense_basic_conv', reuse=None):
    with tf.variable_scope(scope):
        net = slim.batch_norm(x, activation_fn=None, scope='bn')
        net = tf.nn.relu(net, name='activation')
        net = slim.conv2d(inputs=net, num_outputs=out_depth, kernel_size=3,
                          activation_fn=None, normalizer_fn=None, 
                          biases_initializer=None, scope='conv')
        return net

### 构建DenseNet基本单元

In [5]:
def dense_block(x, growth_rate, num_layers, scope='dense_block', reuse=None):
    with tf.variable_scope(scope):
        net = x
        for i in range(num_layers):
            # 将前面所有的输出连接到一起作为下一个基本卷积单元的输入
            out = bn_relu_conv(net, growth_rate, scope='block%d'%i)
            net = tf.concat([net, out], axis=-1)
        return net

### 构建transition层

In [6]:
def transition(x, out_depth, scope='transition', reuse=None):
    in_depth = x.get_shape().as_list()[-1]
    
    with tf.variable_scope(scope):
        net = slim.batch_norm(x, activation_fn=None, scope='bn')
        net = tf.nn.relu(net, name='activation')
        net = slim.conv2d(net, out_depth, 1, activation_fn=None, 
                          normalizer_fn=None, biases_initializer=None, scope='conv')
        return net

### 构建DenseNet整体结构

In [7]:
def DenseNet(x, num_classes, growth_rate=32, block_layers=[6, 12, 24, 16], is_training=None, scope='DenseNet', reuse=None, verbose=False):
    with tf.variable_scope(scope, reuse=reuse):
        with slim.arg_scope([slim.batch_norm], is_training=is_training):
            
            if verbose:
                print('input {}'.format(x.shape))
            
            with tf.variable_scope('block0'):
                net = slim.conv2d(x, 64, [7, 7], stride=2, 
                                  normalizer_fn=None, activation_fn=None, scope='conv7_7')
                net = slim.batch_norm(net, activation_fn=None, scope='bn')
                net = tf.nn.relu(net, name='activation')
                net = slim.max_pool2d(net, [3, 3], stride=2, scope='max_pool')
                
                if verbose:
                    print('block0: {}'.format(net.shape))
                
            # 循环创建`dense_block`和`transition`
            for i, num_layers in enumerate(block_layers):
                with tf.variable_scope('block%d' % (i + 1)):
                    net = dense_block(net, growth_rate, num_layers)
                    if i != len(block_layers) - 1:
                        current_depth = net.get_shape().as_list()[-1]
                        net = transition(net, current_depth // 2)

                if verbose:
                    print('block{}: {}'.format(i+1, net.shape))

            with tf.variable_scope('block%d' % (len(block_layers) + 1)):
                net = slim.batch_norm(net, activation_fn=None, scope='bn')
                net = tf.nn.relu(net, name='activation')
                net = tf.reduce_mean(net, [1, 2], name='global_pool', keep_dims=True)

                if verbose:
                    print('block{}: {}'.format(len(block_layers) + 1, net.shape))

            with tf.variable_scope('classification'):
                net = slim.flatten(net, scope='flatten')
                net = slim.fully_connected(net, num_classes, activation_fn=None, normalizer_fn=None, scope='logit')

                if verbose:
                    print('classification: {}'.format(net.shape))

                return net

In [8]:
with slim.arg_scope([slim.conv2d], activation_fn=tf.nn.relu, normalizer_fn=slim.batch_norm) as sc:
    conv_scope = sc

In [9]:
is_training = tf.placeholder(tf.bool, name='is_training')

with slim.arg_scope(conv_scope):
    train_out = DenseNet(train_imgs, 10, is_training=is_training, verbose=True)
    val_out = DenseNet(val_imgs, 10, is_training=is_training, reuse=True)

input (64, 96, 96, 3)
block0: (64, 23, 23, 64)
block1: (64, 23, 23, 128)
block2: (64, 23, 23, 256)
block3: (64, 23, 23, 512)
block4: (64, 23, 23, 1024)
Instructions for updating:
keep_dims is deprecated, use keepdims instead
block5: (64, 1, 1, 1024)
classification: (64, 10)


In [10]:
with tf.variable_scope('loss'):
    train_loss = tf.losses.sparse_softmax_cross_entropy(labels=train_labels, logits=train_out, scope='train')
    val_loss = tf.losses.sparse_softmax_cross_entropy(labels=val_labels, logits=val_out, scope='val')

In [11]:
with tf.name_scope('accuracy'):
    with tf.name_scope('train'):
        train_acc = tf.reduce_mean(tf.cast(tf.equal(tf.argmax(train_out, axis=-1, output_type=tf.int32), train_labels), tf.float32))
    with tf.name_scope('val'):
        val_acc = tf.reduce_mean(tf.cast(tf.equal(tf.argmax(val_out, axis=-1, output_type=tf.int32), val_labels), tf.float32))

In [12]:
lr = 0.01

opt = tf.train.MomentumOptimizer(lr, momentum=0.9)

In [ ]:
update_ops = tf.get_collection(tf.GraphKeys.UPDATE_OPS)
with tf.control_dependencies(update_ops):
    train_op = opt.minimize(train_loss)

### 开始训练

In [ ]:
from utils.learning import train_with_bn
train_with_bn(train_op, train_loss, train_acc, val_loss, val_acc, 20000, is_training)


Instructions for updating:
To construct input pipelines, use the `tf.data` module.
